In [ ]:
# install hmmer library
!sudo apt-get install hmmer
!sudo apt-get install hmmer-doc
!sudo apt-get install ncbi-blast+

In [2]:
import pandas as pd

## Get the 3D structure
CSV files of PF00014 domains are based on two different queries.

```
( Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Data Collection Resolution < 3 AND Polymer Entity Sequence Length = [ 50 - 80 ] AND Polymer Entity Mutation Count < 10
```
and
```
 ( Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Data Collection Resolution < 2 AND Polymer Entity Sequence Length = [ 50 - 80 ] AND Polymer Entity Mutation Count < 2
 ```
 the difference are the `Resolution(3Å vs 2Å)`, `Polymer Entity Mutation Count(10 vs 2)` and at the end `grouping the polymer entities with different sequence identity(100% vs 50%)`.

 The stricter criteria, with a resolution of 2Å, a mutation count of less than 2, and a sequence identity of 100%, may lead to a smaller sample size but potentially higher quality data, while the less strict criteria, with a resolution of 3Å, a mutation count of less than 10, and a sequence identity of 50%, could yield a larger sample size but with a risk of including lower quality data. With the stricter rules, we have obtained 14 samples, whereas with the less strict rule, we have collected 28 samples. Additionally, we aim to assess which set of criteria—either the more stringent or the less restrictive—ultimately yields superior results in terms of sample quality and relevance to our research objectives.

Now we can download the tabular CSV files which contain `Entity ID`, `Sequence`, `Auth Asym ID`

In [ ]:
!wget -O strict_seq.csv "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict.csv"
!wget -O not_strict_seq.csv "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/not_strict.csv"

## Clean csv file

In [4]:
def clean_csv_file(path: str, output_file_name: str, string_or_file: str = 'f', save_format: str = 'f') -> str or None:
    """
    Reads and cleans a CSV file, providing options to return the cleaned data as a string or save it into a file.

    Parameters:
        path (str): The path to the CSV file to be cleaned.
        output_file_name (str): The name of the output file. Defaults to "output_seq".
        string_or_file (str): Determines whether to return the cleaned data as a string ('s') or save it into a file ('f'). Defaults to 'f'.
        save_format (str): Determines the format for saving the data into a file. For keys ('k') or Fasta format ('f'). Defaults to 'f'.

    Returns:
        str or None: If the user chooses to get the results as a variable ('s'),
        the cleaned data is returned as a string. If the user chooses to save the
        results into a file ('f'), the cleaned data is saved into a file.
    """
    df = pd.read_csv(path)
    df = df.dropna(subset=['Entity ID'])
    df['Entity ID'] = df['Entity ID'].str.split('_').str[0] + ':' + df['Auth Asym ID']
    df = df.drop(columns=['Auth Asym ID'])
    df = df.reset_index(drop=True)

    # Save the output into a variable
    if string_or_file == 's':
        return '\n'.join(df['Entity ID'].values)

    # Save the output into a file
    elif string_or_file == 'f':
        if save_format == 'f':
            with open(output_file_name + '.fasta', 'w') as file:
                for idx, row in df.iterrows():
                    file.write(f"> {row['Entity ID']}\n{row['Sequence']}\n")
            print('Data saved to', output_file_name + '.fasta')
        elif save_format == 'k':
            with open(output_file_name + '.txt', 'w') as f:
                f.write('\n'.join(df['Entity ID'].values))
            print('Data saved to', output_file_name + '.txt')


In [5]:
clean_csv_file(
    path='strict_seq.csv',
    output_file_name='strict_seqs',
    string_or_file='f',
    save_format='f'
    )

Data saved to strict_seqs.fasta


In [6]:
clean_csv_file(
    path='not_strict_seq.csv',
    output_file_name='not_strict_seqs',
    string_or_file='f',
    save_format='f'
    )

Data saved to not_strict_seqs.fasta


In [7]:
!cat strict_seqs.fasta | head -n 10

> 1AAP:A
VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA
> 1KTH:A
ETDICKLPKDEGTCRDFILKWYYDPNTKSCARFWYGGCGGNENKFGSQKECEKVCAPV
> 1ZR0:B
PTGNNAEICLLPLDYGPCRALLLRYYYDRYTQSCRQFLYGGCEGNANNFYTWEACDDACWRIE
> 3BYB:A
KDRPDFCELPADTGPCRVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCAA
> 3M7Q:B
EAEASICSEPKKVGRCKGYFPRFYFDSETGKCTPFIYGGCGGNGNNFETLHQCRAICRALG


We can use the same function which we used to clean the csv file, to only extract the ids of the sequences. these ids are going to be used as input files in the PDBeFold website to get Multiple Seqence Alignment based on Multiple Structure Alignemt.

In [8]:
# Get sequence ids as .txt file for the PDBeFold input
clean_csv_file(
    path='strict_seq.csv',
    output_file_name='strict_ids',
    string_or_file='f',
    save_format='k'
    )

Data saved to strict_ids.txt


In [9]:
# Get sequence ids as .txt file for the PDBeFold input
clean_csv_file(
    path='not_strict_seq.csv',
    output_file_name='not_strict_ids',
    string_or_file='f',
    save_format='k'
    )

Data saved to not_strict_ids.txt


Files `strict_ids.txt` and `not_strict_ids.txt` are uploaded into the [PDBeFold](https://www.ebi.ac.uk/msd-srv/ssm/cgi-bin/ssmserver) and the results are saved in my Github repository.

In [ ]:
!rm not_strict_seq.csv strict_seq.csv

## Get MSA

In [ ]:
# Getting the multiple sequence alignment from github repo
!wget -O strict_msa.txt "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict_msa.txt"

In [ ]:
!wget -O not_strict_msa.txt "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/not_strict_msa.txt"

In [14]:
!rm strict_ids.txt not_strict_ids.txt strict_seqs.fasta not_strict_seqs.fasta

### Build HMM based on the raw MSA

In [ ]:
# Create an HMM model based on the strict_msa.txt file
!hmmbuild strict_msa_not_clean.hmm strict_msa.txt

In [ ]:
# Create an HMM model based on the not_strict_msa.txt file
!hmmbuild not_strict_msa_not_clean.hmm not_strict_msa.txt

In [17]:
!cat strict_msa_not_clean.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  strict_msa
LENG  57
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Sun May 12 12:59:12 2024
NSEQ  15
EFFN  2.354736
CKSUM 1225978556
STATS LOCAL MSV       -8.8676  0.71902
STATS LOCAL VITERBI   -9.0620  0.71902
STATS LOCAL FORWARD   -4.0485  0.71902
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.60263  2.73612  3.07114  2.69296  2.83763  2.63193  3.88604  3.41319  2.66892  3.03622  4.07894  2.70680  3.44272  3.07560  2.87241  2.77771  2.91062  3.13755  4.62504  2.85507
          2.68622  4.42249  2.77475  2.73061  3.46378  2.40519  3.72518  3.29307  2.67748  2.69379  4.24714  2.90341  2.73739  3.18170  2.89777  2.37911  2.77518  2.98542  4.58501  3.61527
          0.72401  0.99536  1.92687  0.72737  0.66006  0.000

In [18]:
!cat not_strict_msa_not_clean.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  not_strict_msa
LENG  59
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Sun May 12 12:59:13 2024
NSEQ  28
EFFN  3.284668
CKSUM 1929399890
STATS LOCAL MSV       -8.9836  0.71896
STATS LOCAL VITERBI   -9.1716  0.71896
STATS LOCAL FORWARD   -4.2058  0.71896
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.66045  2.92817  3.06444  2.66755  2.75405  2.64152  3.84570  3.40036  2.59333  3.03034  4.07046  2.68786  3.42610  3.06466  2.78686  2.80020  2.93294  3.20393  4.59062  2.86329
          2.68661  4.42268  2.77530  2.73007  3.46397  2.40541  3.72365  3.29271  2.67756  2.69368  4.24660  2.90390  2.73730  3.18114  2.89805  2.37919  2.77491  2.98562  4.58371  3.61546
          1.18984  1.11301  1.00195  1.14078  0.38503  0

Based on the files above, we can observe that the `hmmbuild` command, applied to the `strict_msa.txt` file, cuts the first `5` characters in the sequence while when using the `not_strict_msa.txt` it cuts the first `20` character. This action is taken because there are not enough amino acids to build the Hidden Markov Model (HMM) for that part of the sequence. Therefore, we will trim each sequence and then reapply the `hmmbuild` command.

## Clean raw MSA

In [19]:
def clean_msa(path: str, first_clipping_num, output_file_name: str) -> None:
    """
    Clean MSA file by removing specified number of characters from the beginning of each sequence.

    Args:
        path (str): Path to the input MSA file.
        first_clipping_num (int): Number of characters to remove from the beginning of each sequence.
        output_file_name (str): Name of the output file.

    Returns:
        None

    This function reads a MSA file, extracts the sequence IDs and sequences, removes the specified
    number of characters from the beginning of each sequence, and writes the cleaned sequences to a new file.
    """
    with open(path) as f:
        fastas = f.read().split('\n\n')

    clean_list = []
    for fasta in fastas:
        id = fasta.split()[0]
        sequence = ''.join(fasta.split('\n')[1:])
        clean_list.append((id, sequence))

    with open(output_file_name + '.txt', 'w') as f:
        for item in clean_list:
            f.write(f"{item[0]}\n{item[1][first_clipping_num-1:]}\n")

    print(f'Output saved in {output_file_name}.txt')

In [20]:
clean_msa(
    path='strict_msa.txt',
    first_clipping_num=5,
    output_file_name='clean_strict_msa'
    )

Output saved in clean_strict_msa.txt


In [21]:
clean_msa(
    path='not_strict_msa.txt',
    first_clipping_num=20,
    output_file_name='clean_not_strict_msa'
    )

Output saved in clean_not_strict_msa.txt


In [22]:
# Check the strict_msa.tx file
!cat clean_strict_msa.txt | head -n 20

>PDB:1aap:A
vrevcseqaetgpcrAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCg---
>PDB:1kth:A
etdicklpkdegtcrDFILKWYYDPNTKSCARFWYGGCGGNENKFGSQKECEKVCapv-
>PDB:1zr0:B
naeicllpldygpcrALLLRYYYDRYTQSCRQFLYGGCEGNANNFYTWEACDDACwrie
>PDB:3byb:A
rpdfcelpadtgpcrVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCa---
>PDB:3m7q:B
easicsepkkvgrckGYFPRFYFDSETGKCTPFIYGGCGGNGNNFETLHQCRAICralg
>PDB:3wny:C
rpafcleppyagpgkARIIRYFYNAKAGAAQAFVYGGVRAKRNNFASAADALAACaa--
>PDB:4dtg:K
kpdfcfleedpgicrGYITRYFYNNQTKQCERFKYGGCLGNMNNFETLEECKNICedgh
>PDB:4ntw:B
pafcyedppffqkcgAFVDSYYFNRSRITCVHFFYGQCDVNQNHFTTMSECNRVChg--
>PDB:4u30:X
--acanlpivrgpcrAFIQLWAFDAVKGKCVLFPYGGCQGNGNKFYSEKECREYCg---
>PDB:4u32:X
-hdfclvskvvgrcrASMPRWWYNVTDGSCQLFVYGGCDGNSNNYLTKEECLKKC----


In [23]:
# Check the not_strict_msa.tx file
!cat clean_not_strict_msa.txt | head -n 20

>PDB:1aap:A
-vrevcseqaetgpcrAMISRWYFDVTEGKCAPFFYGGcGG-NRNNFDTEEYCMAVCg---
>PDB:1bun:B
krhpdcdkppdtkicqTVVRAFYYKPSAKRCVQFRYGG-CNgNGNHFKSDHLCRCECleyr
>PDB:1dtx:A
prrklcilhrnpgrcyDKIPAFYYNQKKKQCERFDWSGcGG-NSNRFKTIEECRRTCig--
>PDB:1fak:I
-apdfcleppydgpcrALHLRYFYNAKAGLCQTFYYGGcLA-KRNNFESAEDCMRTC----
>PDB:1g6x:A
-rpdfcleppyagacrARIIRYFYNAKAGLCQTFVYGGcRA-KRNNFKSAEDCLRTCgga-
>PDB:1kth:A
-etdicklpkdegtcrDFILKWYYDPNTKSCARFWYGGcGG-NENKFGSQKECEKVCapv-
>PDB:1tfx:C
-kpdfcfleedpgicrGYITRYFYNNQTKQCERFKYGGcLG-NMNNFETLEECKNICedg-
>PDB:1yc0:I
qtedyclasnkvgrcrGSFPRWYYDPTEQICKSFVYGGcLG-NKNNYLREEECILACrgv-
>PDB:1ylc:B
-rpdfxleppytgpckARIIRYFYNAKAGLXQTFVYGGcRA-KRNNFKSAEDXMRTXg---
>PDB:1yld:B
-rpdfxleppytgpckARIIRYFYNAPDGLXQTFVYGGcRA-KRNNFKSAEDXMRTXg---


In [24]:
!rm not_strict_msa.txt not_strict_msa_not_clean.hmm strict_msa.txt strict_msa_not_clean.hmm

### Build HMM based on clean MSA

In [ ]:
# Create an HMM model based on the clean_strict_msa.txt file
!hmmbuild strict_msa.hmm clean_strict_msa.txt

In [ ]:
# Create an HMM model based on the clean_not_strict_msa.txt file
!hmmbuild not_strict_msa.hmm clean_not_strict_msa.txt

In [28]:
!cat strict_msa.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  clean_strict_msa
LENG  57
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Sun May 12 13:01:29 2024
NSEQ  15
EFFN  2.354736
CKSUM 1789090652
STATS LOCAL MSV       -8.8676  0.71902
STATS LOCAL VITERBI   -9.0620  0.71902
STATS LOCAL FORWARD   -4.0485  0.71902
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.60159  2.72559  3.07564  2.69248  2.83160  2.63525  3.88831  3.41485  2.66881  3.04155  4.07693  2.70448  3.45613  3.07429  2.87209  2.78410  2.91250  3.13968  4.62557  2.84818
          2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503
          0.16761  4.74272  1.92687  0.61958  0.77255 

In [29]:
!cat not_strict_msa.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  clean_not_strict_msa
LENG  59
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Sun May 12 13:01:38 2024
NSEQ  28
EFFN  3.284668
CKSUM 2283207759
STATS LOCAL MSV       -8.9836  0.71896
STATS LOCAL VITERBI   -9.1716  0.71896
STATS LOCAL FORWARD   -4.2058  0.71896
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.66000  2.91479  3.07028  2.66650  2.74526  2.64615  3.84796  3.40234  2.59193  3.03732  4.06766  2.68449  3.44347  3.06275  2.78503  2.80934  2.93592  3.20817  4.59074  2.85415
          2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503
          0.46826  4.99854  1.00195  0.61958  0.77

* In these new files, we can observe that the probabilities start from the first amino acid (AA), indicating that no cutting is performed by the `hmmbuild` command itself.

## Get the negative and postive data from NCBI

In [ ]:
!wget -O negative.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28reviewed%3Atrue%29+NOT+%28xref%3Apfam-PF00014%29%29"
!zcat -f negative.fasta.gz > negative.fasta
!rm negative.fasta.gz

In [ ]:
!wget -O bpti_reviewd.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29+AND+%28reviewed%3Atrue%29%29"
!zcat bpti_reviewd.fasta.gz > bpti_reviewd.fasta
!rm bpti_reviewd.fasta.gz

In [ ]:
# Make blast dataset for the bpti_reviewd.fasta
!makeblastdb -in bpti_reviewd.fasta -dbtype prot

In [ ]:
!cat bpti_reviewd.fasta | wc

   1860    5387  118161


The line of the code below initiates a BLASTP search, a tool for comparing protein sequences. It takes protein sequences from "output_seq.fasta" as the query and compares them against a database specified in "bpti_reviewd.fasta". The results are saved in "bpti_reviewd.blast" using format 7, which is suitable for further analysis.

In [ ]:
!blastp -query output_seq.fasta -db bpti_reviewd.fasta -out bpti_reviewd.blast -outfmt 7

This command below filters a BLAST result file named "bpti_reviewd.blast". It removes comment lines (lines starting with # character), selects entries with sequence identity greater than 98%, and saves the unique identifiers of those entries into a file named "remove.fasta".

In [ ]:
!grep -v "^#" bpti_reviewd.blast | awk '{if ($3 > 98) {print $2}}' | sort -u > remove.fasta

In [ ]:
!cat remove.fasta | head -n 5

sp|G9I929|VKTA_MICTN
sp|O43278|SPIT1_HUMAN
sp|O43291|SPIT2_HUMAN
sp|P00974|BPT1_BOVIN
sp|P00980|VKTHA_DENAN


We are only interested in the ids, to get the ids from the remove.fasta file, we should run the command below. the results would be saved in the `remove.ids` file.

In [ ]:
!cat remove.fasta | cut -d "|" -f 2 > remove.ids

In [ ]:
!wc remove.ids

 27  27 189 remove.ids


Based on the `remove.ids` file, there are 27 sequences which should be removed from the main data.

In [ ]:
def filter_sequences(seq_file_path, ids_file_path, output_file_path):
    """
    Filters sequences from a FASTA file based on a list of excluded sequence IDs and save them in a file.

    Parameters:
    - seq_file_path (str): The file path to the input FASTA file containing sequences to filter.
    - ids_file_path (str): The file path to the input file containing a list of sequence IDs to exclude.
    - output_file_path (str): The file path to save the filtered sequences.

    Returns:
    - None
    """
    # Open the file containing excluded sequence IDs and create a set to store them
    with open(ids_file_path, 'r') as file:
        excluded_ids = {line.strip() for line in file}

    # Open the input FASTA file and extract sequences
    with open(seq_file_path, 'r') as file:
        content = file.read().strip()
        sequences = content.split('>')[1:]

    # Open the output file for writing filtered sequences
    with open(output_file_path, 'w') as outfile:
        for sequence in sequences:
            header = sequence.split('\n', 1)[0]
            seq_id = header.split('|')[1]

            if seq_id not in excluded_ids:
                outfile.write(f'>{sequence}\n')


In [ ]:
filter_sequences('bpti_reviewd.fasta', 'remove.ids', 'pos_filtered.fasta')

In [ ]:
!cat pos_filtered.fasta | grep ">" | wc

    364    3628   37607


We can see that now the number of the sequences are 364.

In [ ]:
!cat negative.fasta | grep ">" | wc

 570891 8385714 74362141
